### Creación de un cliente MCP local con LlamaIndex

Este Jupyter Notebook te guía en la creación de un cliente MCP (Protocolo de Contexto de Modelo) local que puede comunicarse con una base de datos mediante herramientas expuestas por un servidor MCP, completamente en tu equipo. Sigue las celdas en orden para un tutorial fluido e independiente.

In [107]:
# Importa nest_asyncio para permitir la ejecución de bucles de eventos anidados en notebooks
import nest_asyncio
# Aplica el parche para que asyncio funcione correctamente en entornos interactivos como Jupyter
nest_asyncio.apply()

#### Configurar una LLM local

In [108]:
# Importa la clase Ollama para usar modelos LLM locales
from llama_index.llms.ollama import Ollama
# Importa Settings para configurar el modelo globalmente en LlamaIndex
from llama_index.core import Settings

# Crea una instancia del modelo LLM local especificando el modelo y el timeout
llm = Ollama(model="llama3.2", request_timeout=120.0)
# Asigna el modelo creado a la configuración global de LlamaIndex
Settings.llm = llm

#### Inicializar el cliente MCP y crear el agente

In [109]:
# Importa las clases necesarias para crear un cliente MCP y especificar herramientas
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# Inicializa el cliente MCP apuntando al endpoint del servidor MCP local
mcp_client = BasicMCPClient("http://localhost:8000/sse")
# Crea el objeto de herramientas MCP, que puede filtrar o limitar las herramientas disponibles
mcp_tools = McpToolSpec(client=mcp_client)  # También puedes pasar una lista de herramientas permitidas

In [110]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

add_person 
    Inserta un nuevo registro en la tabla 'people' de la base de datos.

    Permite agregar personas a la base de datos proporcionando nombre, edad y profesión.

    Parámetros:
        name (str): Nombre de la persona.
        age (int): Edad de la persona.
        profession (str): Profesión de la persona.

    Retorna:
        str: Mensaje de éxito o error.

    Ejemplo de uso:
        >>> add_person('Cristiano Ronaldo', 41, 'Football Player')
        "Persona 'Cristiano Ronaldo' agregada exitosamente"
    
get_people 
    Lee datos de la tabla 'people' de la base de datos usando una consulta SQL SELECT.

    Permite recuperar registros de la tabla 'people' según la consulta SQL proporcionada. 
    Por defecto, devuelve todos los registros.

    Parámetros:
        query (str, opcional): Consulta SQL SELECT. Por defecto es "SELECT * FROM people".

    Retorna:
        list: Lista de diccionarios con los resultados de la consulta.

    Ejemplo de uso:
        >>> get_peo

### Definir el indicador del sistema

In [111]:
from llama_index.llms.ollama import Ollama

async def get_agent_ollama(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with Our Database software.",
        tools=tools,
        llm=Ollama(model="llama3.1", base_url="http://localhost:11434"),
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

Este mensaje guía al LLM cuando necesita decidir cómo y cuándo llamar a las herramientas.



In [112]:
SYSTEM_PROMPT = """\
Eres un asistente de IA para la llamada a herramientas.

Antes de ayudar a un usuario, necesitas trabajar con herramientas para interactuar con nuestra base de datos.
"""

#### Helper function: `get_agent()`

Crea un `FunctionAgent` conectado con la lista de herramientas MCP y el LLM elegido.

In [113]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

async def get_agent_openrouter(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with Our Database software.",
        tools=tools,
        llm=OpenAI(
            model="openai/gpt-4o-mini",
            api_key="API_KEY_HERE",
            api_base="https://openrouter.ai/api/v1"
        ),        
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

In [114]:
from llama_index.llms.ollama import Ollama

async def get_agent_ollama(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with Our Database software.",
        tools=tools,
        llm=Ollama(
            model="llama3.1:latest",  # o cualquier modelo que tengas en Ollama
            request_timeout=360.0,
        ),        
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

#### Helper function: `handle_user_message()`

Transmite llamadas a herramientas intermedias (para transparencia) y devuelve la respuesta final.

In [115]:
from llama_index.core.agent.workflow import (
    FunctionAgent, 
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

#### Initialize the MCP client and build the agent

Apunte el cliente al punto final SSE de su servidor MCP local (el valor predeterminado se muestra a continuación), cree el agente y configure el contexto del agente.

In [116]:
import os
import asyncio
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# Set environment variables for longer timeouts
os.environ['OLLAMA_REQUEST_TIMEOUT'] = '360'
os.environ['OLLAMA_READ_TIMEOUT'] = '640'

# Create MCP client
mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# Your existing code continues...
agent = await get_agent_ollama(mcp_tool)
agent_context = Context(agent)

In [117]:
while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

User:  Add John Smith who is 30 years old and works as a Software Developer


ResponseError: model requires more system memory (29.7 GiB) than is available (28.3 GiB) (status code: 500)